# 自動走行(カメラあり)

※　ネットワーク環境によっては遅延が発生します。

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson Nanoを認識")
    I2C_BUSNUM = 1
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN":
    print("Jetson AGX Orinを認識")
    I2C_BUSNUM = 7
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN_NANO":
    print("Jetson Orin Nanoを認識")
    I2C_BUSNUM = 7
    MODE = 0

TensorRTに変換したモデルを読み込みます。

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

STEERING_CH = 0
THROTTLE_CH = 1
direction = 0

pwm_front = 0
pwm_back = 0

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]
    
if pwm_front >= pwm_back:
    direction = 0
else:
    direction = 1

SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
INITIAL_VALUE=pwm_stop
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)
    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

cameraクラスのインスタンスを生成します。

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=30)

In [ ]:
from torch2trt import TRTModule

model_trt = TRTModule()

widgetでCameraのOn/Offを調整できるようにします。

In [ ]:
import cv2
import ipywidgets
import threading

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

クルマを走らせる前に、スライドバーのゲインやオフセットを準備して、走行中に調整できるようにしておきましょう。

最後に、以下のセルを実行すると、頂点のx値に基づいてレースカーを操り、レースカーを前進させることができます。

以下にヒントを示します。

* 車が左右にふらつく場合は、ステアリングゲインを下げてください。
* 車が曲がりきれない場合は、ステアリングゲインを上げる。
* 車が右に傾いている場合は、ステアリングバイアスをよりネガティブにする（-0.05のような小さな単位で）。
* 車が左に傾いたら、ステアリング・バイアスをよりポジティブにする（小刻みに +0.05）。

In [ ]:
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets
from utils import preprocess
from jetcam.utils import bgr8_to_jpeg
import datetime
import subprocess
import torch

model_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_time_widget = ipywidgets.Label(description='作成日時：')
load_button = ipywidgets.Button(description='Load')

check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')

########################################
# remap
########################################
def remap(x, oMin, oMax, nMin, nMax):
    """
    args:
      x: input value
      oMin: old minimum value
      oMax: old maximum value
      nMin: new minimum value
      nMax: new maximum value
    """

    #range check
    if oMin == oMax:
        print("Warning: Zero input range")
        return None

    if nMin == nMax:
        print("Warning: Zero output range")
        return None

    #check reversed input range
    reverseInput = False
    oldMin = min( oMin, oMax )
    oldMax = max( oMin, oMax )
    if not oldMin == oMin:
        reverseInput = True

    #check reversed output range
    reverseOutput = False   
    newMin = min( nMin, nMax )
    newMax = max( nMin, nMax )
    if not newMin == nMin :
        reverseOutput = True

    portion = (x-oldMin)*(newMax-newMin)/(oldMax-oldMin)
    if reverseInput:
        portion = (oldMax-x)*(newMax-newMin)/(oldMax-oldMin)

    result = portion + newMin
    if reverseOutput:
        result = newMax - portion

    return result

def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = int(remap(x, 224, 0, pwm_right, pwm_left))
    PCA9685.set_channel_value(STEERING_CH, x)
    return x

network_output_slider = widgets.FloatSlider(description='推論結果', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_value_slider = widgets.FloatSlider(description='PWM値', min=pwm_left, max=pwm_right, value=pwm_center, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='速度(正:前, 負:後)', min=0, max=100, value=0, step=1, orientation='vertical')

run_button = ipywidgets.Button(description='Run')
stop_button = ipywidgets.Button(description='Stop')
count = 0
model_load = False
#steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
#steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
#throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

def check_left(change):
    global pwm_left,STEERING_CH
    PCA9685.set_channel_value(STEERING_CH, pwm_left)

def check_center(change):
    global pwm_center,STEERING_CH
    PCA9685.set_channel_value(STEERING_CH, pwm_center)
    
def check_right(change):
    global pwm_left,STEERING_CH
    PCA9685.set_channel_value(STEERING_CH, pwm_right)
    
def run(change):
    global running, start_time,count,model_load
    if model_load == True:
        if direction == 0:
            pwm_speed = pwm_stop + throttle_slider.value
        else:
            pwm_speed = pwm_stop - throttle_slider.value
        write_log("AIを起動します。Speed(PWM): " + str(pwm_speed))
        count = 0
        start_time = time.time()
        update({'new': camera.value})
        camera.observe(update, names='value') 
        camera.running = True
        running = True
    else:
        write_log("[Error] 先に、モデルを読み込んでくさい。")

def stop(change):
    global running,execute_thread,end_time,start_time,count,pwm_stop
    if running == True:
        camera.unobserve(update, names='value')
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count/int(end_time)
            process_time = int((end_time/count)*1000)
        except:
            fps = -1
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数 " + str(count) + ", 処理時間(平均値): " + str(process_time) + " ms")
        running = False
        #execute_thread.join()
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")
        
def load_model(change):
    global model_trt,model_widget,model_load
    try:
        write_log(model_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_trt.load_state_dict(torch.load(model_widget.value))
        write_log(model_widget.value + "の読込に成功しました。")
        model_load = True
    except:
        write_log("[Error]" + model_widget.value + "の読込に失敗しました。")
        
def model_list(change):
    global model_widget, DIR
    try:
        files = glob.glob('./model_trt/*.pth', recursive=True)
        model_widget.options = files
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        model_time_widget.value = f'作成日時：{s}'
    except:
        model_widget.options = []
model_list("list")

def change_file(change):
    file = model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_time_widget.value = f'作成日時：{s}'
model_widget.observe(change_file, names='value')

def update(change):
    global blocked_slider, robot, count, model_trt
    new_image = change['new'] 
    count += 1
    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[1])
    
    network_output_slider.value = x
    if(x<-1.0):
        x = -1.0
    elif(x>1.0):
        x = 1.0
    
    pwm_x = int(camera.width * (x / 2.0 + 0.5))
    pwm_x = handle(pwm_x)
    steering_value_slider.value = pwm_x
    
    if direction == 0:
        pwm_speed = pwm_stop + throttle_slider.value
    else:
        pwm_speed = pwm_stop - throttle_slider.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_speed) 
    
    
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)

run_button.on_click(run)
stop_button.on_click(stop)
check_left_button.on_click(check_left)
check_center_button.on_click(check_center)
check_right_button.on_click(check_right)
load_button.on_click(load_model)

display(
    ipywidgets.HBox([model_widget,model_time_widget,load_button]),
    widgets.HBox(
        [widgets.VBox([network_output_slider, 
                       steering_value_slider], layout=Layout(align_items='center')
                     ), 
         live_execution_widget,
         throttle_slider]
    ),
    widgets.HBox(),
    ipywidgets.HBox([check_left_button,check_center_button,check_right_button]),
    widgets.HBox([run_button,stop_button]),
    process_widget,
    
)

下記コードを実行してカメラの映像が更新されたタイミングで、update()を読み込む処理を実行します。

### カメラの終了処理(必須)
再度、データセット追加や学習は、'02_inreragtive_regression.ipynb'に戻りおこないます。
その際に、下記セルを実行して、カメラの終了処理を忘れずにおこなってください。

In [ ]:
import time
camera.running = False
time.sleep(1)
camera.cap.release()